#Identifying Game Review Attribution with Alibi Gradients

Authors: Jason Phillips & Peeja Khorashadi

Date: 05/01/2023

CSC 215-01

Summary: Utilizing a dataset of Steam reviews for popular games, we are trying to identify the key attributes/words that contribute to a person recommending the game or not.

Dataset:https://www.kaggle.com/datasets/arashnic/game-review-dataset

Papers Refrenced: https://ieeexplore-ieee-org.proxy.lib.csus.edu/document/9752380 , https://ieeexplore-ieee-org.proxy.lib.csus.edu/document/9950194 , https://www.kaggle.com/code/mehmetlaudatekman/game-review-classification-deep-learning-based , https://github.com/SeldonIO/alibi

#Import and Define
Import necessary libraries and define useful functions

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#!pip install tf-explain
#!pip install --upgrade gensim
#!pip install alibi[tensorflow]

In [2]:
#Import and print out the current version of packages.
%load_ext tensorboard

import tensorflow as tf
#import tf_explain
import numpy as np
import matplotlib.pyplot as plt
import sys
import sklearn as sk
import pandas as pd
import shutil
import os
import imblearn
import io
import requests
import datetime
import string

from alibi.explainers import IntegratedGradients

from scipy.stats import zscore
from scipy import spatial

from collections.abc import Sequence
from collections import Counter
from numpy import where
from matplotlib import pyplot

from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,LSTM
from keras.activations import relu, sigmoid, tanh
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers

from gensim.models import Word2Vec
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from pprint import pprint

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

def to_sequences(seq_size, data):
    x = []
    y = []

    for i in range(len(data)-SEQUENCE_SIZE-1):
        #print(i)
        window = data[i:(i+SEQUENCE_SIZE)]
        after_window = data[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)

print('Python {}'.format(sys.version))
print('Pandas {}'.format(pd.__version__))
print('Numpy {}'.format(np.__version__))
print('Scikit-Learn {}'.format(sk.__version__))
print()
print('Tensor Flow {}'.format(tf.__version__))
print('Keras {}'.format(tf.keras.__version__))
print('Imblearn {}'.format(imblearn.__version__))

#Remove any files in the log folder for tensorboard
!rm -rf "/content/drive/MyDrive/Colab Notebooks/logs/"

Python 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
Pandas 1.5.3
Numpy 1.22.4
Scikit-Learn 1.2.2

Tensor Flow 2.12.0
Keras 2.12.0
Imblearn 0.10.1


In [3]:
#Define the log folder for tensorboard
log_dir = "/content/drive/MyDrive/Colab Notebooks/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#Data Preprocessing (Training Dataset)
Cleaning up the data and make sure we only use what we need.

In [51]:
#Read the review dataset from the training folder
filename_read = "https://raw.githubusercontent.com/JasonTPhillipsJr/CSC215/main/Final%20Project/train.csv"
df_train = pd.read_csv(filename_read, sep=',', header=0)
print("Training data shape: " + str(df_train.shape))
df_train

Training data shape: (17494, 5)


,review_id,title,year,user_review,user_suggestion
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1
...,...,...,...,...,...
17489,25535,EverQuest II,2012.0,Arguably the single greatest mmorp that exists...,1
17490,25536,EverQuest II,2017.0,"An older game, to be sure, but has its own cha...",1
17491,25537,EverQuest II,2011.0,When I frist started playing Everquest 2 it wa...,1
17492,25538,EverQuest II,NaN,cool game. THe only thing that REALLY PISSES M...,1


In [52]:
#Print out the different games used in this review.
#NOTE: Remove the ® symbol from smite and elder scrolls later.
print("Number of titles in the dataset: " + str(df_train['title'].nunique()))
print(df_train['title'].unique())

Number of titles in the dataset: 44
["Spooky's Jump Scare Mansion" 'Sakura Clicker' 'WARMODE'
 'Fractured Space' 'Dota 2' 'Path of Exile' 'War Thunder'
 'Ring of Elysium' 'Team Fortress 2' 'Yu-Gi-Oh! Duel Links' 'SMITE®'
 'Brawlhalla' 'World of Tanks Blitz' 'DCS World Steam Edition'
 'Heroes & Generals' 'The Elder Scrolls®: Legends™' 'Trove' 'Neverwinter'
 'Realm Royale' 'PlanetSide 2' 'Realm of the Mad God' 'Elsword'
 'theHunter Classic' 'Eternal Card Game' 'Black Squad'
 'Freestyle 2: Street Basketball' 'Bless Online'
 'RaceRoom Racing Experience' 'Fallout Shelter'
 'Tactical Monsters Rumble Arena' 'Creativerse' 'Dreadnought'
 'Infestation: The New Z' 'Shop Heroes' 'Robocraft'
 'Business Tour - Board Game with Online Multiplayer'
 'Crusaders of the Lost Idols' 'AdventureQuest 3D' 'Realm Grinder'
 'World of Guns: Gun Disassembly' 'Bloons TD Battles' 'Cuisine Royale'
 'School of Dragons' 'EverQuest II']


In [53]:
#Drop unnecessary columns
df_train = df_train.drop(['review_id', 'year'], axis=1)
print("Training data shape: " + str(df_train.shape))
df_train.head()

Training data shape: (17494, 3)


,title,user_review,user_suggestion
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1
1,Spooky's Jump Scare Mansion,"Best game, more better than Sam Pepper's YouTu...",1
2,Spooky's Jump Scare Mansion,"A littly iffy on the controls, but once you kn...",1
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1


In [54]:
#Check if any values are empty or null
df_train[df_train.isnull().any(axis=1)]

,title,user_review,user_suggestion


In [55]:
#Tokenize the tweets to get individual words per row.
tkr = RegexpTokenizer("[a-zA-Záéíóúñ']+")
df_train['tokenized'] = df_train['user_review'].apply(lambda row: [t.lower() for t in tkr.tokenize(row)])
df_train

,title,user_review,user_suggestion,tokenized
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1,"[i'm, scared, and, hearing, creepy, voices, so..."
1,Spooky's Jump Scare Mansion,"Best game, more better than Sam Pepper's YouTu...",1,"[best, game, more, better, than, sam, pepper's..."
2,Spooky's Jump Scare Mansion,"A littly iffy on the controls, but once you kn...",1,"[a, littly, iffy, on, the, controls, but, once..."
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1,"[great, game, fun, and, colorful, and, all, th..."
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1,"[not, many, games, have, the, cute, tag, right..."
...,...,...,...,...
17489,EverQuest II,Arguably the single greatest mmorp that exists...,1,"[arguably, the, single, greatest, mmorp, that,..."
17490,EverQuest II,"An older game, to be sure, but has its own cha...",1,"[an, older, game, to, be, sure, but, has, its,..."
17491,EverQuest II,When I frist started playing Everquest 2 it wa...,1,"[when, i, frist, started, playing, everquest, ..."
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1,"[cool, game, the, only, thing, that, really, p..."


In [56]:
#Drop rows that have more than 150 words
df_train=df_train[df_train["tokenized"].str.len() <= 150]
df_train=df_train[~df_train["user_review"].str.contains(r'[~@#$%^*/\|<>♥¥™_=+◕ヘ░▄▐☉ʖ]')] 
df_train

,title,user_review,user_suggestion,tokenized
0,Spooky's Jump Scare Mansion,I'm scared and hearing creepy voices. So I'll...,1,"[i'm, scared, and, hearing, creepy, voices, so..."
3,Spooky's Jump Scare Mansion,"Great game, fun and colorful and all that.A si...",1,"[great, game, fun, and, colorful, and, all, th..."
4,Spooky's Jump Scare Mansion,Not many games have the cute tag right next to...,1,"[not, many, games, have, the, cute, tag, right..."
6,Spooky's Jump Scare Mansion,Great game. it's a cute little horror game tha...,1,"[great, game, it's, a, cute, little, horror, g..."
9,Spooky's Jump Scare Mansion,This game with its cute little out of the wall...,1,"[this, game, with, its, cute, little, out, of,..."
...,...,...,...,...
17486,EverQuest II,I have always loved this game! I love the idea...,1,"[i, have, always, loved, this, game, i, love, ..."
17487,EverQuest II,After I had played it extensively for four yea...,1,"[after, i, had, played, it, extensively, for, ..."
17488,EverQuest II,this was the first MMORPG i've ever played but...,0,"[this, was, the, first, mmorpg, i've, ever, pl..."
17492,EverQuest II,cool game. THe only thing that REALLY PISSES M...,1,"[cool, game, the, only, thing, that, really, p..."


#Data Preprocessing (Test Dataset)

In [57]:
#Read the review dataset from the training folder
filename_read = "https://raw.githubusercontent.com/JasonTPhillipsJr/CSC215/main/Final%20Project/test.csv"
df_test = pd.read_csv(filename_read, sep=',', header=0)
print("Test data shape: " + str(df_test.shape))
df_test

Test data shape: (8045, 4)


,review_id,title,year,user_review
0,1603,Counter-Strike: Global Offensive,2015.0,"Nice graphics, new maps, weapons and models. B..."
1,1604,Counter-Strike: Global Offensive,2018.0,I would not recommend getting into this at its...
2,1605,Counter-Strike: Global Offensive,2018.0,Edit 11/12/18I have tried playing CS:GO recent...
3,1606,Counter-Strike: Global Offensive,2015.0,The game is great. But the community is the wo...
4,1607,Counter-Strike: Global Offensive,2015.0,I thank TrulyRazor for buying this for me a lo...
...,...,...,...,...
8040,25198,GUNS UP!,2017.0,Early Access ReviewGuns UP!Positive: Good Idea...
8041,25199,GUNS UP!,2018.0,"After 170 hrs, my review. DO NOT SPEND ANY MON..."
8042,25200,GUNS UP!,2018.0,Pros:-Fun GameplayCons: -Micro %$#*ing transac...
8043,25201,GUNS UP!,2018.0,"Actualy saucy, I definetly suggest to players ..."


In [58]:
#Print out the different games used in this review.
#NOTE: Remove the ® symbol from smite and elder scrolls later.
print("Number of titles in the dataset: " + str(df_test['title'].nunique()))
print(df_test['title'].unique())

Number of titles in the dataset: 20
['Counter-Strike: Global Offensive' 'World of Warships' 'Star Trek Online'
 'Paladins®' 'Shadowverse CCG' 'Tree of Savior (English Ver.)'
 'VEGA Conflict' 'Minion Masters' 'The Lord of the Rings Online™'
 'Fishing Planet' 'Crush Crush' 'Dungeon Defenders II'
 'Governor of Poker 3' 'Digimon Masters Online' 'Shakes and Fidget'
 'Champions Online' 'Magic Duels' 'Aura Kingdom' 'H1Z1' 'GUNS UP!']


In [59]:
#Drop unnecessary columns
#NOTE: DROP THE TITLE I THINK???
df_test = df_test.drop(['review_id', 'year'], axis=1)
print("Training data shape: " + str(df_test.shape))
df_test.head()

Training data shape: (8045, 2)


,title,user_review
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...


In [61]:
#Check if any values are empty or null
df_test[df_test.isnull().any(axis=1)]

,title,user_review


In [62]:
#Tokenize the tweets to get individual words per row.
tkr = RegexpTokenizer("[a-zA-Záéíóúñ']+")
df_test['tokenized'] = df_test['user_review'].apply(lambda row: [t.lower() for t in tkr.tokenize(row)])
df_test

,title,user_review,tokenized
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B...","[nice, graphics, new, maps, weapons, and, mode..."
1,Counter-Strike: Global Offensive,I would not recommend getting into this at its...,"[i, would, not, recommend, getting, into, this..."
2,Counter-Strike: Global Offensive,Edit 11/12/18I have tried playing CS:GO recent...,"[edit, i, have, tried, playing, cs, go, recent..."
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...,"[the, game, is, great, but, the, community, is..."
4,Counter-Strike: Global Offensive,I thank TrulyRazor for buying this for me a lo...,"[i, thank, trulyrazor, for, buying, this, for,..."
...,...,...,...
8040,GUNS UP!,Early Access ReviewGuns UP!Positive: Good Idea...,"[early, access, reviewguns, up, positive, good..."
8041,GUNS UP!,"After 170 hrs, my review. DO NOT SPEND ANY MON...","[after, hrs, my, review, do, not, spend, any, ..."
8042,GUNS UP!,Pros:-Fun GameplayCons: -Micro %$#*ing transac...,"[pros, fun, gameplaycons, micro, ing, transact..."
8043,GUNS UP!,"Actualy saucy, I definetly suggest to players ...","[actualy, saucy, i, definetly, suggest, to, pl..."


In [63]:
#Drop rows that have more than 150 words
df_test=df_test[df_test["tokenized"].str.len() <= 150]
df_test=df_test[~df_test["user_review"].str.contains(r'[~@#$%^*/\|<>♥¥™_=+◕ヘ░▄▐☉ʖ]')] 
df_test

,title,user_review,tokenized
0,Counter-Strike: Global Offensive,"Nice graphics, new maps, weapons and models. B...","[nice, graphics, new, maps, weapons, and, mode..."
3,Counter-Strike: Global Offensive,The game is great. But the community is the wo...,"[the, game, is, great, but, the, community, is..."
15,Counter-Strike: Global Offensive,I don't know what it was like before it went F...,"[i, don't, know, what, it, was, like, before, ..."
16,Counter-Strike: Global Offensive,"Extremely toxic community, you will be lucky t...","[extremely, toxic, community, you, will, be, l..."
20,Counter-Strike: Global Offensive,This game will give you low trust factor after...,"[this, game, will, give, you, low, trust, fact..."
...,...,...,...
8036,GUNS UP!,Early Access Reviewincredibly straight forward...,"[early, access, reviewincredibly, straight, fo..."
8038,GUNS UP!,Early Access ReviewActive Dev Team.Gets update...,"[early, access, reviewactive, dev, team, gets,..."
8040,GUNS UP!,Early Access ReviewGuns UP!Positive: Good Idea...,"[early, access, reviewguns, up, positive, good..."
8043,GUNS UP!,"Actualy saucy, I definetly suggest to players ...","[actualy, saucy, i, definetly, suggest, to, pl..."


#Word Embeddings
Create word embeddings so that we have something to input to our model.

In [ ]:
"""
#This goes after word embedding .-.
from tensorflow.keras.preprocessing import sequence
df_train["tokenized"] = sequence.pad_sequences(df_train["tokenized"], maxlen = 150)
df_train
"""

In [ ]:
def prepare_data(df):
  data = df.copy()
  data = data.drop(["user_review", "title"], axis=1)
  x,y = to_xy(data, "user_suggestion")
  return x,y

In [ ]:
x_train, y_train = prepare_data(df_train)

ValueError: ignored